In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import sys

Using TensorFlow backend.


In [2]:
text = (open(r'Dataset/dataset.txt',encoding='utf-8').read())
text = text.lower()
chars = sorted(list(set(text)))
char_to_int = dict((c,i) for i, c in enumerate(chars))

In [3]:
n_chars = len(text)
n_vocab = len(chars)
print("Total characters:",n_chars)
print("Total Vocab:",n_vocab)

Total characters: 11070
Total Vocab: 57


In [4]:
#Prepare the dataset of input to output pairs of encoded as integers
seq_lenght = 100
dataX = []
dataY = []
for i in range(0,n_chars-seq_lenght, 1):
    seq_in = text[i:i+seq_lenght]
    seq_out = text[i + seq_lenght]
    dataX.append([char_to_int[char]for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns:",n_patterns)

Total Patterns: 10970


In [5]:
# Reshape x to be [samples, times steps, features]
X = numpy.reshape(dataX,(n_patterns, seq_lenght,1))
# Normalize
X = X/float(n_vocab)
# One-Hot Encode the output variable
y = np_utils.to_categorical(dataY)

In [6]:
# Define the LSTM Model
model = Sequential()
model.add(LSTM(256,input_shape = (X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer = "adam")

In [7]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [8]:
model.fit(X,y,epochs=20, batch_size=128,callbacks=callbacks_list)

Epoch 1/20
10970/10970 [==============================] - 19s 2ms/step - loss: 3.1555

Epoch 00001: loss improved from inf to 3.15546, saving model to weights-improvement-01-3.1555.hdf5
Epoch 2/20
10970/10970 [==============================] - 16s 1ms/step - loss: 3.0357

Epoch 00002: loss improved from 3.15546 to 3.03566, saving model to weights-improvement-02-3.0357.hdf5
Epoch 3/20
10970/10970 [==============================] - 16s 1ms/step - loss: 3.0300

Epoch 00003: loss improved from 3.03566 to 3.03001, saving model to weights-improvement-03-3.0300.hdf5
Epoch 4/20
10970/10970 [==============================] - 16s 1ms/step - loss: 3.0233

Epoch 00004: loss improved from 3.03001 to 3.02331, saving model to weights-improvement-04-3.0233.hdf5
Epoch 5/20
10970/10970 [==============================] - 17s 2ms/step - loss: 3.0172

Epoch 00005: loss improved from 3.02331 to 3.01719, saving model to weights-improvement-05-3.0172.hdf5
Epoch 6/20
10970/10970 [==============================

In [10]:
# Load the weights 
filename = "weights-improvement-20-2.8089.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy',optimizer = 'adam')

int_to_char = dict((i,c) for i,c in enumerate(chars))

In [ ]:
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" eloped by them after hurricane maria on energy resilience. their work is a reflection of their effor "
 te tee tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  aoe tee aat  a